In [1]:
import pandas as pd
import numpy as np

from horta_exp.introns.fetch_data import get_intron_events
introns = get_intron_events()

from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [2]:
data = pd.read_pickle('chrom_real_chrom22.pkl')
data.set_index(['gene', 'intron'], inplace=True)
data.sort_index(inplace=True)

# Bonferroni correction

In [3]:
ntests = len(data.index.unique())
print("Number of tests: %d" % ntests)

# Bonferroni within gene-intron
def pval_agg(x):
    return np.clip(np.min(x) * len(x), 0, 1)

def method_agg(x):
    return x[0]
    
def gene_intron_wise(df):
    return data.groupby(level=[0, 1]).agg({
        'lmm-pval': pval_agg,
        'lmm-rank-pval': pval_agg,
        'qep-pval': pval_agg
    })

data = gene_intron_wise(data)

Number of tests: 1671


In [4]:
def method_column(df):
    newdata = pd.DataFrame(columns=('pval', 'method'))

    def append_df(data, df, name):
        pval = df['%s-pval' % name]
        return pd.concat([data, pd.DataFrame({'pval': pval, 'method': name})])
        return data

    newdata = append_df(newdata, df, 'lmm')
    newdata = append_df(newdata, df, 'lmm-rank')
    newdata = append_df(newdata, df, 'qep')
    return newdata

In [5]:
def set_index(data):
    data = method_column(data)
    data.reset_index(inplace=True)
    data['gene'] = data['index'].apply(lambda x: x[0])
    data['intron'] = data['index'].apply(lambda x: int(x[1]))
    del data['index']
    data.set_index(['gene', 'intron'], inplace=True)
    data.sort_index(inplace=True)
    return data

In [6]:
data = set_index(data)

In [7]:
def plot_curve(data, perc=False, exclude=False, lthr=-5, rthr=-0.30103):
    
    nhits = {'lmm-rank':[], 'qep':[]}
    if not exclude:
        nhits['lmm'] = []
    
    pts = np.logspace(lthr, rthr)
    for pt in pts:

        if perc:
            grouped = data.groupby(data['method']).agg({
                'pval': lambda x: 100*sum(x*len(x)<=pt)/float(len(x))
            })
        else:
            grouped = data.groupby(data['method']).agg({
                'pval': lambda x: sum(x*len(x)<=pt)
            })


        nhits['lmm-rank'].append(grouped.loc['lmm-rank', 'pval'])
        if not exclude:
            nhits['lmm'].append(grouped.loc['lmm', 'pval'])
        nhits['qep'].append(grouped.loc['qep', 'pval'])

    p = figure(title="Power", tools=['save,reset'])
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = 'P-value threshold'
    if perc:
        p.yaxis.axis_label = 'Percentage of hits'
    else:
        p.yaxis.axis_label = 'Number of hits'
    p.line(pts, nhits['lmm-rank'], color='green', legend='lmm-rank')
    if not exclude:
        p.line(pts, nhits['lmm'], color='blue', legend='lmm')
    p.line(pts, nhits['qep'], color='red', legend='qep')
    p.legend.location = "bottom_right"
    return p

In [8]:
lthr = -5
rthr = -1
p1 = plot_curve(data, False, lthr=lthr, rthr=rthr)
p2 = plot_curve(data, False, True, lthr=lthr, rthr=rthr)
show(gridplot([[p1,p2]], plot_width=400, plot_height=400))

In [9]:
def plot_data(data, title='title'):
    print(title)
    lthr = -5
    rthr = -1
    p1 = plot_curve(data, True, lthr=lthr, rthr=rthr)
    p2 = plot_curve(data, True, True, lthr=lthr, rthr=rthr)
    show(gridplot([[p1,p2]], plot_width=400, plot_height=400, title=title))

In [10]:
g = introns.groupby(level=[0, 1])
g = g.agg({
    'ntri': {
        'max': 'max',
        'min': 'min',
    },
    'nsuc': {
        'max': 'max',
        'min': 'min',
    }
})
g.head()

ntri      nsuc    
                         max  min  max min
gene            intron                    
ENSG00000000419 5        951   29  148   5
                6        675    8    9   0
                7        863   28   50   0
                8       1243  104  571   4
ENSG00000000457 1       1309   73   83   0

In [11]:
g2 = introns.groupby(level=[0, 1])
g2 = g2.agg({
    'ntri': {
        'median': 'median',
    },
    'nsuc': {
        'median': 'median',
    }
})
g2.head()

ntri   nsuc
                       median median
gene            intron              
ENSG00000000419 5         347     39
                6         218      1
                7         291      6
                8         449     95
ENSG00000000457 1         432     12

In [12]:
g['ntri']['max'].head()

gene             intron
ENSG00000000419  5          951
                 6          675
                 7          863
                 8         1243
ENSG00000000457  1         1309
Name: max, dtype: int32

In [13]:
for quant in [0.25, 0.5, 0.75, 1.]:
    d = data.loc[g['ntri']['max'] <= g['ntri']['max'].quantile(quant)]
    plot_data(d, title='Quantile: %f; Number of gene-introns: %d' % (quant, len(d.index.unique())))

Quantile: 0.250000; Number of gene-introns: 371


Quantile: 0.500000; Number of gene-introns: 797


Quantile: 0.750000; Number of gene-introns: 1262


Quantile: 1.000000; Number of gene-introns: 1671


In [14]:
for quant in [0.25, 0.5, 0.75, 1.]:
    d = data.loc[g2['ntri']['median'] <= g2['ntri']['median'].quantile(quant)]
    plot_data(d, title='Quantile: %f; Number of gene-introns: %d' % (quant, len(d.index.unique())))

Quantile: 0.250000; Number of gene-introns: 361


Quantile: 0.500000; Number of gene-introns: 795


Quantile: 0.750000; Number of gene-introns: 1278


Quantile: 1.000000; Number of gene-introns: 1671


In [15]:
for quant in [0.75, 0.5, 0.25, 0.]:
    d = data.loc[g2['ntri']['median'] > g2['ntri']['median'].quantile(quant)]
    plot_data(d, title='Quantile: %f; Number of gene-introns: %d' % (quant, len(d.index.unique())))

Quantile: 0.750000; Number of gene-introns: 393


Quantile: 0.500000; Number of gene-introns: 876


Quantile: 0.250000; Number of gene-introns: 1310


Quantile: 0.000000; Number of gene-introns: 1671
